# Process :  de Boyer Montegut MLD climatology
_process already re-gridded MLD climatology_

- **Original**: `/local/data/artemis/observations/MLD_deBoyerMontegut/processed/mld_DT02_1x1_clim.nc`
- **Processed**: `/local/data/artemis/observations/MLD_deBoyerMontegut/processed/mld_1x1_clim_deBoyer_198201-201712.nc`
- **Variable**: mld
- **Temporal**: Monthly climatology (tiled for 36 years)
- **Spatial**: 1x1 degree, interpolated from 2x2

Laste update : 20 September 2019 L. Gloege

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
# Observations directory
dir_obs = '/local/data/artemis/observations'

In [3]:
# Read data
ds = xr.open_dataset(f'{dir_obs}/MLD_deBoyerMontegut/processed/mld_DT02_1x1_clim.nc', decode_cf=False)

In [4]:
ds['mld']

<xarray.DataArray 'mld' (time: 12, ylat: 180, xlon: 360)>
[777600 values with dtype=float32]
Coordinates:
  * time     (time) float64 14.0 45.0 73.0 104.0 ... 257.0 287.0 318.0 348.0
  * xlon     (xlon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * ylat     (ylat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Attributes:
    long_name:      mixed layer depth (estimated by kriging of mld_smth)
    units:          meters
    _FillValue:     -9999.0
    missing_value:  -9999.0
    min_value:      10.267677
    max_value:      772.3705
    mask_value:     1000000000.0

# Tile the climatology for N years

In [5]:
# Tile the climatology to nyears
nyears = 36
ds_tiled = np.tile(ds['mld'],[nyears, 1, 1])

In [6]:
print(f'ds_tiled shape = {np.shape(ds_tiled)}')

ds_tiled shape = (432, 180, 360)


In [7]:
# Create a date vector
dates = pd.date_range(start='1982-01-01T00:00:00.000000000', 
                      end='2017-12-01T00:00:00.000000000',freq='MS')+ np.timedelta64(14, 'D')

In [8]:
print(f'dates shape = {np.shape(dates)}')

dates shape = (432,)


In [11]:
ds_out = xr.Dataset(
    {
    'mld':(['time','lat', 'lon'], ds_tiled, {'long_name':'mixed layer depth',
                                             'units':'meters'}),
    },

    coords={
    'lat': (['lat'], ds['ylat'], {'units':'degrees_north'}),
    'lon': (['lon'], ds['xlon'], {'units':'degrees_east'}),
    'time': (['time'], dates)
    })

In [12]:
ds_out.info()

xarray.Dataset {
dimensions:
	lat = 180 ;
	lon = 360 ;
	time = 432 ;

variables:
	float32 mld(time, lat, lon) ;
		mld:long_name = mixed layer depth ;
		mld:units = meters ;
	float64 lat(lat) ;
		lat:units = degrees_north ;
	float64 lon(lon) ;
		lon:units = degrees_east ;
	datetime64[ns] time(time) ;

// global attributes:
}

## Save to NetCDF file

In [14]:
# Save tiled climatology
ds_out.to_netcdf(f'{dir_obs}/MLD_deBoyerMontegut/processed/mld_1x1_clim_deBoyer_198201-201712.nc')

/home/gloege/miniconda3/envs/dev/lib/python3.6/site-packages/xarray/coding/times.py:244: FutureWarning: the 'box' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'box'
  dates = pd.to_datetime(dates, box=False)
